In [0]:
-- CONTRACT ANALYSIS
-- -- Assemble 

CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.assembled') (
  path STRING,
  doc_info STRING,
  preamble STRING,
  vs_results STRING,
  vs_preamble_results STRING,
  other_folder_docs BIGINT,
  other_preambles STRING,
  other_doc_infos STRING
);

MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.assembled') AS target
USING (
  WITH doc_info_search_results AS (
    SELECT
      doc.path,
      concat_ws('\n \n ##### \n \n', collect_list(vs.combined_doc_info)) AS vs_combined_doc_info
    FROM IDENTIFIER(:catalog || '.' || :schema || '.doc_info') doc
    CROSS JOIN LATERAL (
      SELECT combined_doc_info
      FROM vector_search(
        index => :catalog || '.' || :schema || '.doc_info_index',
        query_text => substring(doc.combined_doc_info, 0, :vs_char_limit),
        query_type => 'HYBRID',
        num_results => 5
      )
    ) vs
    GROUP BY doc.path
  ),
  preamble_search_results AS (
    SELECT
      f.path,
      concat_ws('\n \n ##### \n \n', collect_list(vs_preamble.preamble)) AS vs_preamble_results
    FROM IDENTIFIER(:catalog || '.' || :schema || '.flat') f
    CROSS JOIN LATERAL (
      SELECT preamble
      FROM vector_search(
        index => :catalog || '.' || :schema || '.flat_index',
        query_text => substring(f.preamble, 0, :vs_char_limit),
        query_type => 'HYBRID',
        num_results => 5
      )
    ) vs_preamble
    GROUP BY f.path
  )
  SELECT
    base.path,
    base.preamble,
    doc.combined_doc_info AS doc_info,
    vs.vs_combined_doc_info AS vs_results,
    preamble_vs.vs_preamble_results AS vs_preamble_results,
    count(DISTINCT rel.other_preamble) AS other_folder_docs,
    concat_ws('\n \n ##### \n \n', collect_list(DISTINCT rel.other_preamble)) AS other_preambles,
    concat_ws('\n \n ##### \n \n', collect_list(DISTINCT rel.other_combined_doc_info)) AS other_doc_infos
  FROM IDENTIFIER(:catalog || '.' || :schema || '.flat') base
  LEFT JOIN (
    SELECT
      o.other_path AS path,
      t.preamble AS other_preamble,
      doc.combined_doc_info AS other_combined_doc_info
    FROM (
      SELECT
        path,
        vendor_folder_path,
        preamble
      FROM IDENTIFIER(:catalog || '.' || :schema || '.flat')
      LATERAL VIEW OUTER explode(vendor_folder_paths) AS vendor_folder_path
    ) t
    JOIN (
      SELECT
        path AS other_path,
        vendor_folder_path AS other_vendor_folder_path
      FROM IDENTIFIER(:catalog || '.' || :schema || '.flat')
      LATERAL VIEW OUTER explode(vendor_folder_paths) AS vendor_folder_path
    ) o
      ON t.vendor_folder_path = o.other_vendor_folder_path
      AND t.path <> o.other_path
    LEFT JOIN IDENTIFIER(:catalog || '.' || :schema || '.doc_info') doc
      ON t.path = doc.path
  ) rel
  ON base.path = rel.path
  LEFT JOIN IDENTIFIER(:catalog || '.' || :schema || '.doc_info') doc
  ON base.path = doc.path
  LEFT JOIN doc_info_search_results vs
  ON base.path = vs.path
  LEFT JOIN preamble_search_results preamble_vs
  ON base.path = preamble_vs.path
  GROUP BY base.path, base.preamble, doc.combined_doc_info, vs.vs_combined_doc_info, preamble_vs.vs_preamble_results
) AS source
ON target.path = source.path
WHEN MATCHED THEN UPDATE SET
  doc_info = source.doc_info,
  preamble = source.preamble,
  vs_results = source.vs_results,
  vs_preamble_results = source.vs_preamble_results,
  other_folder_docs = source.other_folder_docs,
  other_preambles = source.other_preambles,
  other_doc_infos = source.other_doc_infos
WHEN NOT MATCHED THEN INSERT (
  path,
  doc_info,
  preamble,
  vs_results,
  vs_preamble_results,
  other_folder_docs,
  other_preambles,
  other_doc_infos
) VALUES (
  source.path,
  source.doc_info,
  source.preamble,
  source.vs_results,
  source.vs_preamble_results,
  source.other_folder_docs,
  source.other_preambles,
  source.other_doc_infos
);